In [ ]:
import pandas as pd
import numpy as np
from time import strftime

### Load data

In [ ]:
data = pd.read_csv("carpetas_completa_julio2021.csv")
data.head()

### 1. Pruebas para la calidad de ddatos.

#### Revisión del tipo de atos y casteos correspondientes: Muhas veces los datos deberían sr un tipo de datos específico pero pudieron haberse guarado como otro tipo de datos y para su mejor manejo es importante realizar las conversiones de tipo de datos respectivas.

#### Encontrar y eliminar registros duplicados: Para cualquier análisis es importante remver registros duplicados para no sesgar frecuencias o introducir basura.

#### Análisis de correlaciones: Es importante revisar que columnas tienen una fuerte correlación entre ellas, para posteriores fases del proyecto considerar dejar solo una de las columnas que tenga correlación alta con otra(s).

#### Encontrar y darle el tratamiento respectivo a los datos faltantes (nulos): Es importante para la calidad de los datos revisar si hay datos faltantes y analizar  si es posible inferirlos o eliminarlos.

#### Análisis de outliers: Es importante evitar sesgos en estadísticas básicas y encontrar el comportamiento más normal de los datos ypara ello es importante identificar los posibles outliers y analizar si es conveniente removerlos o tratarlos por separado.

#### Consistencia en los valores de las variables categóricas. Es importante revisar la consistencia e identificar y unificar casos como: la variable género tiene  los siguientes valores para la opción "Femenino": F, fem, f,femenino.

#### Identificar comportamientos entre las fechas de inicio y las fechas de los hechos, hacer un análisis para identificar el tiempo promedio entre que ocurren y se inicia la investigación para identificar inconsistencias.

#### Hacer un análisis para identificar si los datos están balanceados.

#### Análisis de series de tiempo (para casos específicos de interés), el análisis de series de tiempo ayuda también para revisar elcomportamiento de los datos.

### Limpieza de datos.

In [ ]:
data.dtypes

In [ ]:
data.isnull().sum()

In [ ]:
data.shape

In [ ]:
for x in data:
    print(x)
    print(data[x].value_counts(dropna = False))
    print('\n')

In [ ]:
data.drop_duplicates(inplace = True)
data['fecha_hechos'] = pd.to_datetime(data['fecha_hechos'])

In [ ]:
data['fecha_hechos'].min()

In [ ]:
data['fecha_hechos'].max()

In [ ]:
data.delito.unique()

In [ ]:
data.categoria_delito.unique()

In [ ]:
data.head()

### Delitos a la alza y baja en la CDMX

##### es importante quitar registros que no tienen fecha de los hechos para realizar el análisis. Sería posible (dependiendo si el negocio y el análisis/modelo a realizar lo permiten) inferir las fechas de hecho con respecto a las fechas de inicio.

In [ ]:
data_fechas = data[data['fecha_hechos'].notnull()]
data_fechas.shape

##### nos quedamos con las columnas de interés

In [ ]:
data_delito = data_fechas['ao_hechos','delito']

##### dado que para identificar si los delitos van a la alza o a la baja actualmente, se requiere analizar sólo los últimos años (la cantidad de años dependerá también del tipo de estudio a realizar)

In [ ]:
data_delito = data_delito[(data_delito['ao_hechos']>=2018)&(data_delito['ao_hechos']!=2021)]
del data_fecha

##### obtenemos la frecuencia por delito

In [ ]:
df_delitos = pd.DataFrame(data_delito['delito'].value_counts()).reset.index()
df_delitos.columns = ['delito', 'count']

In [ ]:
df_delitos['count'].describe(percentiles = [.05,.1,.2,.25,.4,.5,.6,.75,.9,.95])

##### obtenemos los delitos con muy pocas ocurrencias (no es posible saber si los delitos van a la alza o a la baja)

In [ ]:
no_valid = df_delitos[df_delitos['count']<=2]['delito']

##### quitamos los delitos con pocas ocurrencias

In [ ]:
data_delito = data_delito[~data_delito.delito.isin(no_valid)]
data_d = data_delito.copy()
del df_delitos, data_delito
df_date = data_d[['ao_hechos']]

##### identiicamos que delittos ocurrieron en que año

In [ ]:
data_dumm = pd.get_dummies(data_d[['delito']])
del data_d
data_dumm.columns = [x.split('delito_')[1] for x in data_dumm]
data_dumm.shape

#### agregamos la fecha

In [ ]:
data_dumm = pd.concat([df_date, data_dumm], axis = 1)
del df_date

###### obtenemos el número de ocurrencia de delitos por año, la frecuencia (año, mes, etc)  debe ser determinada al tipo de negocio

In [ ]:
data_dumm_gb = data_dumm.groupby('ao_hechos').sum()
del data_dumm
data_dumm_gb

#### obtenemos las diferencias porcentuales de ocurrencia de cada delito entre cada año para determinar que delitos van a la alza y cales a la baja

In [ ]:
data_pct = data_dumm_gb.pct_change()
data_pct = data_pct.T.reset_index()
data_pct

In [ ]:
baja_nan = data_pct[data_pct[2020.0].isnull()]['index']
baja_signo = data_pct[(data_pct[2020.0]<0)&(data_pct[2021.0]<0)]['index']
baja = list(baja_nan) + list(baja_signo)
baja = np.unique(np.array(baja))

In [ ]:
alza = list(data_pct[(data_pct[2020.0]>0)&(data_pct[2021.0]>0)]['index'])

##### los delitos que van a la alza los últimos años son

In [ ]:
alza

In [ ]:
len(alza)

##### los delitos que van a la baja los últimos años son

In [ ]:
baja

In [ ]:
len(baja)

### 3. Alcaldía con más y con menos delitos

#### obtenemos el número de delitos por alcaldía

In [ ]:
alcaldia = pd.DataFrame(data['alcaldia_hechos'].value_counts()).reset_index()
alcaldia.columns = ['alcaldia','count']
alcaldia

#### el top 5 de alcaldías con más delitos es :

In [ ]:
alcaldia.head()

#### las calcaldías con un solo delito son

In [ ]:
alcaldia[alcaldia['count']==1]['alcaldia'].unique()

#### 4. Tendencias estacionales en la ocurrencia de delitos (mes, quincena, semana, dia de la semana)

##### filtramos por los registros que si tienen fecha de hechos. Sería posible (dependiendo si el negocio y el análisis/modelo a realizar lo permiten) inferir las fechas de hecho con respecto a las fechas de inicio.

In [ ]:
data_fechas = data[data['fecha_hechos'].notnull()][['ao_hechos','mes_hechos','fecha_hechos','delito']]
data_fechas.shape

In [ ]:
data_fechas['mes'] = data_fechas['fecha_hechos'].map(lambda x: x.month)
data_fechas['dia'] = data_fechas['fecha_hechos'].map(lambda x: x.day)
data_fechas['dia_semana'] = data_fechas['fecha_hechos'].map(lambda x: x.weekday()) #0 monday
data_fechas['semana'] = data_fechas['fecha_hechos'].map(lambda x: int(x.strftime("%U")))
data_fechas['quincena'] = data_fechas['week'].map(lambda x: round(x/2))
data_fechas.head()

#### Nos quedamos con las columnas de interés

In [ ]:
df_dates = data_fechas[['delito', 'ao_hechos','mes','quincena','semana','dia_semana','dia']]

#### obtenemos el top 5 de delitos que mas ocurren 

In [ ]:
top5_delitos = ['VIOLENCIA FAMILIAR', 'FRAUDE', 'ROBO DE OBJETOS', 'AMENAZAS', 'ROBO A NEGOCIO SIN VIOLENCIA']

#### obtenemos el top 5 de los años, meses, quincenas, semanas y días con más ocurrencias de delitos

In [ ]:
for col in [x for x in df_dates if 'delito'!=x]:
    display(pd.DataFrame(df_dates[col].value_counts()).reset_index().head())

#### obtenemos el top 5 de los años, meses, quincenas, semanas y días con más ocurrencias de delitos para cada uno de los delitos más ocurridos

In [ ]:
for delito in top5_delitos:
    aux = df_dates[df_dates['delitos']==delito]
    print(delito)
    for col in [x for x in df_dates if 'delito'!=x]:
        display(pd.DataFrame(aux[col].value_counts()).reset_index().head())

### 5. Delitos que caracterizan a cada alcaldía

##### filtramos por los delitos que si tienen alcladía

In [ ]:
data_alcaldia = data[data['alcaldia_hechos'].notnull()]

In [ ]:
# función para obtener los delitos principales en cada alcaldia
def get_principal(data, col_alcaldia, col_delito, i, delitos_omitidos = []):
    df_worker = data.copy()
    df_alcaldia_delitos = pd.DataFrame()
    delitos = []
    counts = []
    opciones = []
    if len(delitos_omitidos) != 0:
        df_worker = df_worker[~df_worker[col_delito].isin(delitos_omitidos)]
    for alcaldia in df_worker[col_alcaldia].unique():
        aux  = df_worker[df_worker[col_alcaldia]==alcaldia]
        if len(aux[col_delito].value_counts().keys()) <= i:
            i = 0
        delito = aux[col_delito].value_counts().keys()[i]
        delitos.append(delito)
        counts.append(aux[col_delito].value_counts().values[i])
        opciones.append(len(aux[col_delito].value_counts().keys()))
    df_alcaldia_delitos['alcaldia'] = df_worker[col_alcaldia].unique()
    df_alcaldia_delitos[col_delito] = delitos
    df_alcaldia_delitos[f'count_{col_delito}'] = counts
    df_alcaldia_delitos['n_opciones'] = opciones
    return df_alcaldia_delitos

In [ ]:
# función para obtener el delito característico (delito que es frecuente en esa alcaldia y en el resto es menos frecuente)
def get_caracteristico(data):
    df_worker = data.copy()
    dict_delito = {}
    for delito in df_worker['delito'].unique():
        aux = df_worker[df_worker['delito']==delito]
        max_delito = aux['count_delito'].max()
        dict_delito[delito] = max_delito
    df_worker['caracteristico'] = df_worker[['delito', 'count_delito']].apply(lambda x: x[0] if x[1] == dict_delito[x[0]] else 'otro', axis = 1)
    df_worker['n_opciones'] = df_worker['n_opciones'] - 1
    return df_worker

In [ ]:
# función para obtener las alcaldías que solo tienen un delito en toda la historia y las alcaldías que no tienen un delito que predomine
def get_diferentes(data, data_2):
    df_worker = data.copy()
    data2 = data_2.copy()
    list_no_validas = []
    list_una_opcion = []
    for alcaldia in df_worker['alcaldia_hechos'].unique()
        aux = df_worker[df_worker['alcaldia_hechos']==alcaldia]
        if aux.shape[0] != 1:
            opciones_delitos = data2[data2['alcaldia']==alcaldia]['n_opciones'].values[0]
            if aux.shape[0] == opciones_delitos:
                list_no_validas.append(alcaldia)
            elif opciones_delitos == 1:
                list_una_opcion.append(alcaldia)
        else:
            list_una_opcion.append(alcaldia)
    return list_no_validas, list_una_opcion

In [ ]:
df_delitos = get_principal(data_alcaldia, 'alcaldia_hechos', 'delito', 0)
df_delitos

In [ ]:
list_no_validas, list_una_opcion = get_diferentes(data_alcaldia, df_delitos)

In [ ]:
len(list_no_validas)

In [ ]:
len(list_una_opcion)

In [ ]:
df_alcaldia_delitos = df_delitos[~df_delitos['alcaldia'].isin(list_no_validas + list_una_opcion)].reset_index(drop = True)

In [ ]:
df_caract = get_caracteristico(df_alcaldia_delitos)

In [ ]:
ant_shape = 158
actual_shape = 126
i = 1

In [ ]:
while (actual_shape != 0) or (actual_shape != ant_shape):
    ant_shape = df_caract[df_caract['caracteristico']=='otro'].shape[0]
    delitos_omitidos = df_caract['caracteristico'].unique()
    alcaldias_no_caract = df_caract[(df_caract['caracteristico'] == 'otro')&(df_caract['n_opciones']>0)]['alcaldia'].unique()
    df_no_caracteristico = data_alcaldia[data_alcaldia['alcaldia'].isin(alcaldias_no_caract)]
    df_alcaldia_delitos = get_principal(df_no_caracteristico, 'alcaldia_hechos', 'delito', i, delitos_omitidos)
    i = i + 1
    df_caract_new = get_caracteristico(df_alcaldia_delitos)
    df_caract = pd.concat([df_caract['caracteristico']!='otro'], df_caract_new)
    actual_shape = df_caract[df_caract['caracteristico']== 'otro'].shape[0]

In [ ]:
df_caract

##### alcaldia con su delito caracteristico

In [ ]:
df_caracteristico = df_caract[['alcaldia','caracteristico']]
df_caracteristico

##### alcaldias con una sola opción caracteristico

In [ ]:
df_una_opcion = data_alcaldia[data_alcaldia['alcaldia_hechos'].isin(list_una_opcion)][['alcaldia_hechos', 'delito']].drop_duplicates().reset_index(drop = True)
df_una_opcion

In [ ]:
# alcaldias con un delito principal
df_delitos

### 6. Indicador del nivel de inseguridad

In [ ]:
data_colonia = data[(data['alcaldia_hechos'].notnull())&(data['colonia_hechos'])]

##### el indicador será para los últimos 10 años

In [ ]:
data_colonia = data_colonia[data_colonia['ao_hechos']>=2010.0]
data_colonia = data_colonia[['delito', 'categoria_delito', 'alcaldia_hechos', 'colonia_hechos']]
data_colonia

In [ ]:
alta_severidad_df = data_colonia[(data_colonia['delito'].str.contains('CON VIOLENCIA'))|(data_colonia['categoria_delito']!='DELITO DE BAJO IMPACTO')]
alta_severidad_freq_df = pd.DataFrame(alta_severidad_df['colonia_hechos'].value_counts()).reset_index()

In [ ]:
baja_severidad_df = data_colonia[~data_colonia['colonia_hechos'].isin(alta_severidad_df['colonia_hechos'].unique())]
baja_severidad_freq_df = pd.DataFrame(baja_severidad_df['colonia_hechos'].value_counts()).reset_index()

In [ ]:
index_4 = list(alta_severidad_freq_df[alta_severidad_freq_df['colonia_hechos']>alta_severidad_freq_df['colonia_hechos'].mean()]['index'])
index_3 = list(alta_severidad_freq_df[alta_severidad_freq_df['colonia_hechos']<=alta_severidad_freq_df['colonia_hechos'].mean()]['index'])
index_2 = list(baja_severidad_freq_df[baja_severidad_freq_df['colonia_hechos']>baja_severidad_freq_df['colonia_hechos'].mean()]['index'])
index_1 = list(baja_severidad_freq_df[baja_severidad_freq_df['colonia_hechos']<=baja_severidad_freq_df['colonia_hechos'].mean()]['index'])

In [ ]:
data_colonia['nivel_inseguridad'] = data_colonia['colonia_hechos'].map(lambda x: 'rojo' if x in index_4 else ('naranja' if x in index_3 else ('amarillo' if x in index_2 else 'verde')))

In [ ]:
data_colonia['nivel_inseguridad'].value_counts()

#### indicador

In [ ]:
data_colonia